In [50]:
import pandas as pd
from decimal import Decimal

In [162]:
# read all values as a string

df = pd.read_csv("/Users/pedrosanhueza/EXOXY/Administrative Documents/Jobs/geoaire/Planilla.csv", dtype=str)

df

,Fecha,"MP2,5 (µg/m³)",Velocidad del viento (m/s),Altura de \nMezcla (m)
0,44562,25.6,"3,0",124.207
1,44562.04167,7,"2,6",78.278
2,44562.08333,5.4,"1,8",51.695
3,44562.125,4,"1,4",50
4,44562.16667,11.5,"1,2",50
...,...,...,...,...
8317,44926.79167,10.1,"3,7",140.479
8318,44926.83333,11.8,"3,2",64.357
8319,44926.875,12.5,"3,1",77.685
8320,44926.91667,10.8,"2,4",77.483


In [167]:
# replace comma with dots

df = df.applymap(lambda x: str(x).replace(',', '.'))

df

,Fecha,"MP2,5 (µg/m³)",Velocidad del viento (m/s),Altura de \nMezcla (m)
0,44562,25.6,3.0,124.207
1,44562.04167,7,2.6,78.278
2,44562.08333,5.4,1.8,51.695
3,44562.125,4,1.4,50
4,44562.16667,11.5,1.2,50
...,...,...,...,...
8317,44926.79167,10.1,3.7,140.479
8318,44926.83333,11.8,3.2,64.357
8319,44926.875,12.5,3.1,77.685
8320,44926.91667,10.8,2.4,77.483


In [168]:
# convert values to decimals

df = df.applymap(lambda x: Decimal(x) if pd.notna(x) else x)

df

,Fecha,"MP2,5 (µg/m³)",Velocidad del viento (m/s),Altura de \nMezcla (m)
0,44562,25.6,3.0,124.207
1,44562.04167,7,2.6,78.278
2,44562.08333,5.4,1.8,51.695
3,44562.125,4,1.4,50
4,44562.16667,11.5,1.2,50
...,...,...,...,...
8317,44926.79167,10.1,3.7,140.479
8318,44926.83333,11.8,3.2,64.357
8319,44926.875,12.5,3.1,77.685
8320,44926.91667,10.8,2.4,77.483


In [176]:
df['Test'] = df['MP2,5 (µg/m³)'] * df['Velocidad del viento (m/s)']

df

,Fecha,"MP2,5 (µg/m³)",Velocidad del viento (m/s),Altura de \nMezcla (m),Test
0,44562,25.6,3.0,124.207,76.80
1,44562.04167,7,2.6,78.278,18.2
2,44562.08333,5.4,1.8,51.695,9.72
3,44562.125,4,1.4,50,5.6
4,44562.16667,11.5,1.2,50,13.80
...,...,...,...,...,...
8317,44926.79167,10.1,3.7,140.479,37.37
8318,44926.83333,11.8,3.2,64.357,37.76
8319,44926.875,12.5,3.1,77.685,38.75
8320,44926.91667,10.8,2.4,77.483,25.92


In [53]:
flux = pd.DataFrame()
flux['MP2,5 (g/m³)']    = df['MP2,5 (µg/m³)'] / 1000 / 1000
flux["Vd,min (m/s)"]    = df['Vd,min (cm/s)'] / 100
flux["Vd,max (m/s)"]    = df['Vd,max (cm/s)'] / 100
flux["Vd (m/s)"]        = df['Vd (cm/s)'] / 100
flux["fmin,t (g/m2*h)"] = flux['MP2,5 (g/m³)'] * flux["Vd,min (m/s)"] * 3600
flux["fmax,t (g/m2*h)"] = flux['MP2,5 (g/m³)'] * flux["Vd,max (m/s)"] * 3600
flux["ft (g/m2*h)"]     = flux['MP2,5 (g/m³)'] * flux["Vd (m/s)"] * 3600

flux

,"MP2,5 (g/m³)","Vd,min (m/s)","Vd,max (m/s)",Vd (m/s),"fmin,t (g/m2*h)","fmax,t (g/m2*h)",ft (g/m2*h)
0,0.0000256000000000,0.0000270000000000,0.0004275000000000,0.0002250000000000,0.0000024883200000,0.0000393984000000,0.0000207360000000
1,0.0000070000000000,0.0000270000000000,0.0004275000000000,0.0002250000000000,0.0000006804000000,0.0000107730000000,0.0000056700000000
2,0.0000054000000000,0.0000180000000000,0.0002445000000000,0.0001350000000000,0.0000003499200000,0.0000047530800000,0.0000026244000000
3,0.0000040000000000,0.0000090000000000,0.0000630000000000,0.0000450000000000,0.0000001296000000,0.0000009072000000,0.0000006480000000
4,0.0000115000000000,0.0000090000000000,0.0000630000000000,0.0000450000000000,0.0000003726000000,0.0000026082000000,0.0000018630000000
...,...,...,...,...,...,...,...
8317,0.0000101000000000,0.0000330000000000,0.0005235000000000,0.0002550000000000,0.0000011998800000,0.0000190344600000,0.0000092718000000
8318,0.0000118000000000,0.0000270000000000,0.0004275000000000,0.0002250000000000,0.0000011469600000,0.0000181602000000,0.0000095580000000
8319,0.0000125000000000,0.0000270000000000,0.0004275000000000,0.0002250000000000,0.0000012150000000,0.0000192375000000,0.0000101250000000
8320,0.0000108000000000,0.0000180000000000,0.0002445000000000,0.0001350000000000,0.0000006998400000,0.0000095061600000,0.0000052488000000


In [54]:
# running time: 70 seconds

resuspension = pd.DataFrame()
resuspension['% Resuspensión']  = df['% Resuspensión']
resuspension['fmin,t (g/m2*h)'] = flux["fmin,t (g/m2*h)"]
resuspension['fmax,t (g/m2*h)'] = flux["fmax,t (g/m2*h)"]
resuspension['ft (g/m2*h)']     = flux["ft (g/m2*h)"]
resuspension['At min (g/m2*h)'] = 0
resuspension['At max (g/m2*h)'] = 0
resuspension['At (g/m2*h)']     = 0
resuspension['Rmin (g/m2*h)']   = 0
resuspension['Rmax (g/m2*h)']   = 0
resuspension['Rt (g/m2*h)']     = 0

rows = []
for fila in range(resuspension.shape[0]-1):
# for fila in range(2):

    resuspension['Rmin (g/m2*h)']   = ( resuspension['At min (g/m2*h)'].shift(1) + resuspension['fmin,t (g/m2*h)'] ) * resuspension['% Resuspensión'] / 100
    resuspension['At min (g/m2*h)'] = resuspension['At min (g/m2*h)'].shift(1) + resuspension['fmin,t (g/m2*h)'] - resuspension['Rmin (g/m2*h)']
    resuspension['Rmax (g/m2*h)']   = ( resuspension['At max (g/m2*h)'].shift(1) + resuspension['fmax,t (g/m2*h)'] ) * resuspension['% Resuspensión'] / 100
    resuspension['At max (g/m2*h)'] = resuspension['At max (g/m2*h)'].shift(1) + resuspension['fmax,t (g/m2*h)'] - resuspension['Rmax (g/m2*h)']
    resuspension['Rt (g/m2*h)']     = ( resuspension['At (g/m2*h)'].shift(1) + resuspension['ft (g/m2*h)'] ) * resuspension['% Resuspensión'] / 100
    resuspension['At (g/m2*h)']     = resuspension['At (g/m2*h)'].shift(1) + resuspension['ft (g/m2*h)'] - resuspension['Rt (g/m2*h)']

    row = {}
    row['Rmin (g/m2*h)']   = resuspension['Rmin (g/m2*h)'].iloc[fila+1]
    row['At min (g/m2*h)'] = resuspension['At min (g/m2*h)'].iloc[fila+1]
    row['Rmax (g/m2*h)']   = resuspension['Rmax (g/m2*h)'].iloc[fila+1]
    row['At max (g/m2*h)'] = resuspension['At max (g/m2*h)'].iloc[fila+1]
    row['Rt (g/m2*h)']     = resuspension['Rt (g/m2*h)'].iloc[fila+1]
    row['At (g/m2*h)']     = resuspension['At (g/m2*h)'].iloc[fila+1]

    rows.append(row)


In [35]:
pd.set_option('display.float_format', lambda x: '%.16f' % x)

In [61]:
resuspension2 = pd.DataFrame(rows)

flux_neto = pd.DataFrame()

flux_neto['Fmin (g/m2*h)'] = flux['fmin,t (g/m2*h)'] - resuspension2['Rmin (g/m2*h)']
flux_neto['Fmax(g/m2*h)']  = flux['fmax,t (g/m2*h)'] - resuspension2['Rmax (g/m2*h)']
flux_neto['Ft (g/m2*h)']   = flux['ft (g/m2*h)']     - resuspension2['Rt (g/m2*h)']

flux_neto

,Fmin (g/m2*h),Fmax(g/m2*h),Ft (g/m2*h)
0,0.0000024577020000,0.0000389136150000,0.0000204808500000
1,0.0000006504089400,0.0000103217611500,0.0000054288225000
2,0.0000003334303359,0.0000045206211577,0.0000024977089125
3,0.0000001077686809,0.0000006391050404,0.0000004952642788
4,0.0000003453127506,0.0000023036426648,0.0000016836383146
...,...,...,...
8317,0.0000001576328686,0.0000033233023005,0.0000012414303785
8318,0.0000000969389895,0.0000022903568969,0.0000014333720114
8319,0.0000005254914233,0.0000088485150910,0.0000047948561806
8320,0.0000000139094806,-0.0000008035281617,-0.0000000497435048


In [83]:
cambio_concentracion = pd.DataFrame()

cambio_concentracion['C (ug/m3*h)']         = df['MP2,5 (µg/m³)']
cambio_concentracion['H (m)']               = df['Altura de Mezcla (m)']
cambio_concentracion['F (ug/m2*h)']         = flux_neto['Ft (g/m2*h)'] * 1000 * 1000
cambio_concentracion['Mtotal (ug/m2*h)']    = cambio_concentracion['C (ug/m3*h)'] * cambio_concentracion['H (m)']
cambio_concentracion['Iunit (%)']           = cambio_concentracion.apply(lambda row:
    (row['F (ug/m2*h)'] * 100 / row['Mtotal (ug/m2*h)'])
    if row['F (ug/m2*h)'] > 0
    else row['F (ug/m2*h)'] * 100 / (row['Mtotal (ug/m2*h)'] + row['F (ug/m2*h)']),
    axis=1)
cambio_concentracion['Itotal (%)']          = (cambio_concentracion['F (ug/m2*h)'] * tc) / (cambio_concentracion['F (ug/m2*h)'] * tc / 100 + cambio_concentracion['Mtotal (ug/m2*h)'])
cambio_concentracion['ΔC (ug/m3*h)']        = cambio_concentracion['C (ug/m3*h)'] / (1 - cambio_concentracion['Itotal (%)'] / 100) - cambio_concentracion['C (ug/m3*h)']

cambio_concentracion

,C (ug/m3*h),H (m),F (ug/m2*h),Mtotal (ug/m2*h),Iunit (%),Itotal (%),ΔC (ug/m3*h)
0,25.6000000000000014,124.2069999999999936,20.4808499999999967,3179.6992000000000189,0.6441128141932418,0.0321952720160311,0.0082446440216728
1,7.0000000000000000,78.2780000000000058,5.4288224999999999,547.9460000000000264,0.9907586696499290,0.0495134055645829,0.0034676553437745
2,5.4000000000000004,51.6950000000000003,2.4977089124999994,279.1530000000000200,0.8947455024663892,0.0447172698352856,0.0024158128566594
3,4.0000000000000000,50.0000000000000000,0.4952642788124999,200.0000000000000000,0.2476321394062499,0.0123800741181926,0.0004952642788121
4,11.5000000000000000,50.0000000000000000,1.6836383146303124,575.0000000000000000,0.2928066634139674,0.0146381900908989,0.0016836383146295
...,...,...,...,...,...,...,...
8317,10.0999999999999996,140.4790000000000134,1.2414303784766776,1418.8378999999999905,0.0874962797706967,0.0043746226069331,0.0004418562128414
8318,11.8000000000000007,64.3569999999999993,1.4333720114452284,759.4125999999999976,0.1887474623735804,0.0094364825626103,0.0011136100280034
8319,12.5000000000000000,77.6850000000000023,4.7948561806201289,971.0625000000000000,0.4937742092419519,0.0246826166423403,0.0030860888077608
8320,10.8000000000000007,77.4830000000000041,-0.0497435047984759,836.8164000000000442,-0.0059447283675029,-0.0002972196329203,-0.0000320996249492


In [125]:
cambio_concentracion['ΔC (ug/m3*h)'].sum()

-2.7019364958644627

In [ ]:
user_input = 3
A = [5,4,4,2,3]
B = [3,1,1,5,3]

# Initialize the sum
C = user_input
D = 0
result = 0

# Calculate the sum dynamically based on the pattern
for a,b in zip(A,B):
    D = C * b + a
    print(f'{D} = {C} * {b} + {a}')
    C = C + D
    print(C)
    result += D

print(result)


In [ ]:
resuspension

In [ ]:
columns = zip(
    resuspension['% Resuspensión'][1:],
    resuspension['fmin,t (g/m2*h)'][1:]
)

In [ ]:
At_min = 0

for Resuspensión,fmin in columns:
    Rmin = ( At_min + fmin ) * Resuspensión / 100
    print(f'Rmin: {Rmin} = ({At_min} + {fmin}) * {Resuspensión} / 100')
    At_min = At_min + fmin - Rmin
    print(At_min, "\n")

print(At_min, Rmin)


In [121]:
a = 0.0001
b = 0.0002

print(a,b)

0.0001 0.0002


In [118]:
a * b

2e-08

In [128]:
a = 0.0001
b = 0.0002

a *= 1000
b *= 1000

print(a,b)

0.1 0.2


In [122]:
(a * 1000 * b * 1000) / 1000 / 1000

2e-08

In [129]:
a*b

0.020000000000000004

In [136]:
from decimal import Decimal, getcontext

getcontext().prec = 20

a = Decimal('0.1')
b = Decimal('0.2')

a*b

Decimal('0.02')